In [ ]:
! pip install scikit-learn

In [ ]:
! pip install numpy

In [ ]:
! pip install joblib

In [ ]:
import numpy as np # Import the NumPy library and abbreviate it as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
import joblib

# Read data from a CSV file:


In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
# This code allows uploading files from the local system to Colab for later file reading
column_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
# Handle the case where the CSV file has no column names.

df = pd.read_csv("iris.csv", names=column_names, header=None) # Read a CSV file
print(df.head()) # View the first 5 rows of the data

Saving iris.csv to iris (1).csv
   sepal_length  sepal_width  petal_length  petal_width      species
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [ ]:
len(df)

150

# Data preprocessing to ensure the data format is suitable for model training:
Here, the `species` column of the Iris dataset, which contains string categories (e.g., *Iris-setosa*), is converted into numerical labels.  

`LabelEncoder()` is a label encoding tool from the scikit-learn library that converts categorical data (such as strings) into numerical values, making it easier for machine learning models to process.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Select features and target variables
X = df.iloc[:, :-1].values  # Select all columns except the last one
# Here, X selects the first 4 columns (excluding the *species* column), because *species* is the target label, not a feature.
y = df.iloc[:, -1].values   # Select the last column of all rows, the categorical column (*species*)
# `iloc` is a way to select data based on index position in Pandas

# Convert categories to numerical values.
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
print("original categories:", y) # View the original categories
print("converted categories:", y_encoded)  # View the converted categories


original categories: ['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versico

# Data normalization:

`StandardScaler` is a class, and `StandardScaler()` is an instance. `StandardScaler().fit(x_tr)` is calling the `fit()` method of the `StandardScaler()` instance.  
`scale` is also a class, and `scale.transform(x_tr)` is calling the `transform()` method of the class.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("The first five rows of normalised data:\n", X_scaled[:5])


The first five rows of normalised data:
 [[-0.90068117  1.03205722 -1.3412724  -1.31297673]
 [-1.14301691 -0.1249576  -1.3412724  -1.31297673]
 [-1.38535265  0.33784833 -1.39813811 -1.31297673]
 [-1.50652052  0.10644536 -1.2844067  -1.31297673]
 [-1.02184904  1.26346019 -1.3412724  -1.31297673]]


# Set random seed to make all further calculations reproducible
Set the random seed to ensure that all subsequent calculations are reproducible.

In [ ]:
seed = 42

# Split the data into training and testing sets：

In [ ]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(X_scaled, y_encoded, test_size=0.20, random_state=seed)

In Python, the commonly used train_test_split() function from scikit-learn is used to split datasets.

X: Feature data (e.g., molecular fingerprints, SMILES representation, etc.).

y: Target variable (e.g., logBB_class).

test_size=0.2: Indicates that 20% of the data will be used for the test set, and 80% will be used for the training set. You can adjust this ratio based on your needs, for example, test_size=0.3 means 30% of the data is used for testing.

random_state=42: Ensures consistency in data splitting (by setting a fixed random seed to make sure the dataset is split the same way every time).

# Create folds for cross-validation:


In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed) # cross-validation

In [ ]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(x_tr, y_tr)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   3   4   6   8   9  10  11  13  14  15  16  17  18  19  20  22
  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  42  43
  44  46  47  48  49  52  53  54  56  57  58  59  60  61  62  63  65  67
  68  69  70  72  74  75  76  77  78  79  81  82  83  84  85  87  88  89
  90  91  92  94  95  97  98  99 101 103 104 105 106 107 108 109 111 112
 113 114 115 116 117 119]
TEST: [  2   5   7  12  21  23  24  41  45  50  51  55  64  66  71  73  80  86
  93  96 100 102 110 118]

Fold_2
TRAIN: [  0   1   2   4   5   6   7   9  11  12  13  14  15  16  17  18  19  21
  22  23  24  26  29  30  31  32  35  36  37  38  40  41  42  43  44  45
  47  48  49  50  51  53  54  55  56  57  58  59  60  61  63  64  65  66
  67  68  69  70  71  72  73  74  75  78  79  80  83  85  86  88  90  91
  92  93  94  95  96  97  98  99 100 102 106 107 108 109 110 111 112 113
 114 115 116 117 118 119]
TEST: [  3   8  10  20  25  27  28  33  34  39  46  52  62  76  77  81  82  84
  87

# Search for the best tuning parameters and build the model:  
Use a grid search dictionary for hyperparameter optimization.  
Hyperparameter optimization refers to finding the optimal combination of hyperparameters to ensure the model performs best on the validation set or test set.

In [ ]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3],
              "n_estimators": [100, 250, 500]}

shape[0] represents the number of rows (samples).
shape[1] represents the number of columns (features).

print(x_tr.shape)     # Output: (5, 3) → 5 rows, 3 columns
print(x_tr.shape[0])  # Output: 5  → Number of samples (rows)
print(x_tr.shape[1])  # Output: 3  → Number of features (columns)

`n_estimators` represents the number of decision trees used in the random forest.

In [ ]:
# setup model building  Create GridSearchCV for hyperparameter search.
iris = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)
#This line of code uses grid search to perform hyperparameter optimization on the `RandomForestClassifier` and searches for the best parameter combination in parallel.

In [ ]:
# run model building
iris.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
45 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_p

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [0, 0, 0, 1],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [ ]:
iris.best_params_

{'max_features': 1, 'n_estimators': 100}

In [ ]:
iris.best_score_

np.float64(0.925)

In [ ]:
iris.cv_results_

{'mean_fit_time': array([7.13253021e-04, 1.06835365e-03, 5.90515137e-04, 5.19418716e-04,
        4.82177734e-04, 5.18703461e-04, 4.67920303e-04, 4.81176376e-04,
        4.56953049e-04, 3.20804024e-01, 6.54187012e-01, 7.86147165e-01]),
 'std_fit_time': array([7.14063605e-05, 8.55999335e-04, 3.98158742e-05, 2.73126563e-05,
        2.02229556e-05, 8.32212132e-05, 2.87476260e-05, 4.25410906e-05,
        3.87215848e-05, 8.43504362e-02, 7.08182658e-02, 1.30449800e-01]),
 'mean_score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.01549816,
        0.03192277, 0.03397307]),
 'std_score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.00408558,
        0.00148456, 0.00409733]),
 'param_max_features': masked_array(data=[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
              mask=[False, False, False, False, False, False, False, False,

In [ ]:
iris.cv_results_['mean_test_score']

array([  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
       0.925, 0.925, 0.925])

In [ ]:
iris.cv_results_['params']

[{'max_features': 0, 'n_estimators': 100},
 {'max_features': 0, 'n_estimators': 250},
 {'max_features': 0, 'n_estimators': 500},
 {'max_features': 0, 'n_estimators': 100},
 {'max_features': 0, 'n_estimators': 250},
 {'max_features': 0, 'n_estimators': 500},
 {'max_features': 0, 'n_estimators': 100},
 {'max_features': 0, 'n_estimators': 250},
 {'max_features': 0, 'n_estimators': 500},
 {'max_features': 1, 'n_estimators': 100},
 {'max_features': 1, 'n_estimators': 250},
 {'max_features': 1, 'n_estimators': 500}]

# Save model:

In [ ]:
joblib.dump(iris, "iris.pkl", compress=3)
# joblib.dump(): Used to save an object (such as a trained model) to a file for later loading and use, without the need to retrain the model.
# iris: The object to be saved, usually a trained machine learning model (e.g., RandomForestRegressor() or GridSearchCV).
# "iris.pkl": The file path where the object will be saved, with the .pkl extension (Pickle format), indicating a serialized Python object.
# compress=3: An optional parameter that specifies the compression level, ranging from 0 (no compression) to 9 (maximum compression). A value of 3 indicates moderate compression, reducing the file size and improving storage efficiency.


['iris.pkl']

# Predict test set compounds:

In [ ]:
# Load the saved model.
scale = joblib.load("iris.pkl")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_ts = scaler.fit_transform(x_ts)

In [ ]:
# predict
pred_rf = iris.predict(x_ts)

In [ ]:
pred_rf

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1,
       0, 2, 2, 2, 2, 2, 0, 0])

# calc statistics for test set preditions

In [ ]:
accuracy_score(y_ts, pred_rf)
# Calculate the classification accuracyAccuracy）

0.9666666666666667

In [ ]:
y_ts

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [ ]:
matthews_corrcoef(y_ts, pred_rf)
# Calculate the Matthews correlation coefficient (MCC)

np.float64(0.9515873026942034)

In [ ]:
cohen_kappa_score(y_ts, pred_rf)
# Calculate Cohen's Kappa score

np.float64(0.95)

# applicability domain estimates

In [ ]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = iris.predict_proba(x_ts)
# `predict_proba` is a method of scikit-learn classification models that returns the probability of each class for the input sample `x_ts`.
# `x_ts` is the test data.

In [ ]:
# probablity
pred_prob

array([[0.  , 0.98, 0.02],
       [0.97, 0.03, 0.  ],
       [0.  , 0.02, 0.98],
       [0.  , 0.98, 0.02],
       [0.  , 0.99, 0.01],
       [1.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.24, 0.76],
       [0.  , 0.89, 0.11],
       [0.  , 1.  , 0.  ],
       [0.  , 0.2 , 0.8 ],
       [0.99, 0.01, 0.  ],
       [1.  , 0.  , 0.  ],
       [0.99, 0.01, 0.  ],
       [1.  , 0.  , 0.  ],
       [0.01, 0.75, 0.24],
       [0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  ],
       [0.  , 0.98, 0.02],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.72, 0.28],
       [1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  ],
       [0.08, 0.01, 0.91],
       [0.  , 0.11, 0.89],
       [0.  , 0.21, 0.79],
       [0.  , 0.03, 0.97],
       [0.99, 0.01, 0.  ],
       [0.99, 0.01, 0.  ]])

In [ ]:
# setup threshold
threshold = 0.8
# In classification problems, setting the threshold is mainly used to determine the decision boundary for the prediction results.
# For example, in a binary classification problem, the `predict_proba` method returns the probability of a certain class. We can set a **threshold** to decide the final classification result.

In [ ]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold

In [ ]:
da

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True])

In [ ]:
# calc statistics
accuracy_score(np.asarray(y_ts)[da], pred_rf[da])
# `accuracy_score` is a function in the scikit-learn library that calculates the accuracy of a classification task

1.0

In [ ]:
np.asarray(y_ts)[da]

array([1, 0, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 2, 0, 0, 2, 2, 2,
       2, 0, 0])

In [ ]:
pred_rf[da]

array([1, 0, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 2, 0, 0, 2, 2, 2,
       2, 0, 0])

In [ ]:
pred_rf

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1,
       0, 2, 2, 2, 2, 2, 0, 0])

In [ ]:
matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

np.float64(1.0)

In [ ]:
cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

np.float64(1.0)

In [ ]:
# calc coverage
sum(da) / len(da)

np.float64(0.8333333333333334)

# Let's try to analyse which variables are the most important in the model

In [ ]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=iris.best_params_["n_estimators"],
                           max_features=iris.best_params_["max_features"],
                           random_state=seed)
rf.fit(x_tr, y_tr)

RandomForestClassifier(max_features=1, random_state=42)

In [ ]:
imp = rf.feature_importances_

In [ ]:
imp

array([0.16729865, 0.12383881, 0.35855938, 0.35030316])

In [ ]:
indices = np.argsort(imp)[::-1]
# `np.argsort()` is a sorting function provided by NumPy that returns the indices of the elements sorted in ascending order, rather than the sorted values.
# This part uses Python's slicing operation to reverse the order of the array.
# Since `np.argsort(imp)` generates indices sorted in ascending order, we use `[::-1]` to reverse it into indices sorted in descending order.

print("Feature ranking:")

# print top 4 features
for i in range(4):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 2 (0.358559)
2. feature 3 (0.350303)
3. feature 0 (0.167299)
4. feature 1 (0.123839)
